In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import altair as alt
import pandas as pd
from datetime import datetime, timedelta



import requests, json, os, warnings, re
import pandas as pd
import datetime


### Long-interval

In [ ]:
#define the ticker symbol
tickerSymbol = 'TSLA'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2020-01-1', end='2020-09-12')

#see your data
tickerDf.tail()


In [ ]:
tickerData.calendar

In [ ]:
tickerData.recommendations

In [ ]:
priceData = tickerDf.Open


In [ ]:
priceData = priceData.asfreq(pd.infer_freq(priceData.index))


In [ ]:
priceData 

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(priceData)
for year in range(priceData.index[0].year, priceData.index[-1].year+1):
    plt.axvline(datetime(year,1,1), color='k', linestyle='--', alpha=0.2)
plt.title("%s Price Data"%tickerSymbol, fontsize=20)

### Short-interval

In [ ]:
from datetime import datetime
from alpha_realtime_stock import AlphaRealtimeStock
import time
import timeit
import requests
from bs4 import BeautifulSoup
import re

def get_crumbs():
    """
    get crumb  from yahoo finance
    returns crumb
    """
    url = 'https://finance.yahoo.com/quote'
    header = {'Connection': 'keep-alive',
              'Expires': '-1',
              'Upgrade-Insecure-Requests': '1',
              'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) \
               AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
              }

    website = requests.get(url, headers=header)
    soup = BeautifulSoup(website.text, 'lxml')
    crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))
    return crumb[0]

crumb = get_crumbs()
ticker_list = ['TSLA']
stock_collection = {}
for ticker in ticker_list:
    stock = AlphaRealtimeStock(ticker, crumb)
    stock_collection[ticker] = stock
# while True:
#     print(datetime.now().strftime("%H:%M:%S"))
    for ticker in ticker_list:
        stock = stock_collection[ticker]
        stock.fetch_webdata()
#         df = stock.get_Stoch_RSI_Indicator()
#         print(df.tail())
#     end = timeit.default_timer()
#     time.sleep(5)

In [ ]:
stock.fetch_webdata().tail()

#### Using API

In [ ]:
params={}

params['function']='TIME_SERIES_INTRADAY'
params['symbol']='TSLA'
params['interval']='5min'
params

In [ ]:
def data_request(params):
    url = "https://www.alphavantage.co/query"
    params['apikey'] = os.environ['ALPHA_PAT']

    return requests.get(url, params)

In [ ]:
def data_request(params):
    url = "https://www.alphavantage.co/query"
    params['apikey'] = os.environ['ALPHA_PAT']

    return requests.get(url, params)
                        
def Get_TimeSeries(stock, period):
    """
    A wrapper function to obtain the the time series of a the desired stock from the Alpha Vantage API.
    
    Parameters:
    -----------
    stock: the desired stock symbol (e.g., "GOOG", "AMD", "FB", "INTC").
    period: "daily", "weekly", "monthly"
    
    Returns:
    --------
    A pandas dataframe with the columns `open`, `high`, `low`, `close`, and `volume`.
    """
    
    # Process here the parameter `period`
    # which affects the function parameter in the API
    
    # TODO

    params={}
    if period=='daily':
        params['function'] = 'TIME_SERIES_DAILY'
        series='Time Series (Daily)'
    elif period=='weekly':
         params['function'] = 'TIME_SERIES_WEEKLY'
         series='Weekly Time Series'
    elif period=='monthly':
         params['function'] = 'TIME_SERIES_MONTHLY'
         series='Monthly Time Series'
    
 
    
    # Now let's create a dictionary to hold the parameters of 
    # our request.
    
    # TODO
    params['symbol'] = stock

    # Requesting the data
    
    # TODO
    r=data_request(params) 
    
    # Check if the request was successful, if not raise an
    # exception
    
    # TODO
        
    
    
    # Create the pandas dataframe (see: pd.read_json() )
    
    # TODO
    json = r.json()
   
    data = pd.DataFrame.from_dict(json[series], orient="index")
    data=data.reset_index()
    data=data.rename(columns={"index": "date", "1. open": "open","2. high": "high","3. low": "low","4. close":"close","5. volume":"volume"})

    data.stock= json['Meta Data']['2. Symbol']
    


    
    return data

In [ ]:
stock_data=Get_TimeSeries('TSLA', 'daily')
stock_data

In [ ]:
stock_data.columns

In [ ]:
def Plot_TimeSeries(data):
    
    title_price='Stock price of '+data.stock
    title_volume= 'Volume of '+data.stock
    stock_color = alt.condition("datum.open <= datum.close",
                                     alt.value("#06982d"),
                                     alt.value("#ae1325"))
    
    base = alt.Chart(data,title=title_price).encode(
        alt.X('date:T',
              axis=alt.Axis(
                  title='Date'
              )
        ),
        color=stock_color
    )

    rule = base.mark_rule().encode(
        alt.Y(
            'low:Q',
            title='Price',
            scale=alt.Scale(zero=False),
        ),
        alt.Y2('high:Q')
    )

    bar = base.mark_bar().encode(
        alt.Y('open:Q'),
        alt.Y2('close:Q')
    )
    price_plot=rule + bar
    volume_plot=alt.Chart(data,title=title_volume).mark_bar().encode(
        alt.X('date:T',
              axis=alt.Axis(
                  title='Date'
              )),
        alt.Y('volume:Q')
   ).properties(height=100)
    
    return(price_plot & volume_plot)

In [ ]:
Plot_TimeSeries(stock_data)

### Modeling

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [ ]:
# using different model

In [ ]:
df1 = yf.download('TSLA','2019-01-29','2020-09-11',interval='1d')

In [ ]:
df1

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Scaled TESLA stocks from August 2014 to August 2017')
plt.xlabel('Days')
plt.ylabel('Scaled value of stocks')
plt.plot(df1['Open'], label='Stocks data')
plt.legend()
plt.show()

In [ ]:
df1 = df1.reset_index()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(df1.shape[0]),(df1['Low']+df1['High'])/2.0)
plt.xticks(range(0,df1.shape[0],50),df1['Date'].loc[::50],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.show()

In [ ]:
# long term prediciton

In [ ]:
df = df1['Open'].values
print("Total No. of days of TSLA's Stocks :",df.shape[0])

In [ ]:
dataset_train = np.array(df[:int(df.shape[0]*0.8)])
dataset_test = np.array(df[int(df.shape[0]*0.8)-50:])
print('Training stock :',dataset_train.shape[0],'days')
print('Test stock :',dataset_test.shape[0],'days')

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train.reshape(-1, 1))

In [ ]:
dataset_test = scaler.transform(dataset_test.reshape(-1, 1))


In [ ]:
def create_dataset(df,n):
    '''
    Function to prepare data for the LSTM
    
    '''
    
    x = []
    y = []
  
    for i in range(n, df.shape[0]):
        x.append(df[i-n:i, 0])
        y.append(df[i, 0])
    x = np.array(x)
    y = np.array(y)
    return x,y

In [ ]:
x_train, y_train = create_dataset(dataset_train,50)
x_test, y_test = create_dataset(dataset_test,50)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
print('Training data shape',x_train.shape)
print('Test data shape',x_test.shape)



In [ ]:
model = Sequential()

model.add(LSTM(units=96, 
               return_sequences=True,
               input_shape=(x_train.shape[1], 1)))

model.add(Dropout(0.2))

model.add(LSTM(units=96, 
               return_sequences=True))

model.add(Dropout(0.2))

model.add(LSTM(units=96, 
               return_sequences=True))

model.add(Dropout(0.2))

model.add(LSTM(units=96))

model.add(Dropout(0.2))

model.add(Dense(units=1))

In [ ]:

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout   ## importing layers of Neural network
from keras.callbacks import ModelCheckpoint     ## imporing modelcheck pointer
from keras.losses import MSE                  ## importing error function
from keras.optimizers import Adam              ## importing optimizer
from keras.activations import relu

In [ ]:
model.summary()


In [ ]:
model.compile(loss=MSE, optimizer=Adam())


In [ ]:
checkpointer = ModelCheckpoint(filepath='tesla.stock_prediction.hdf5',save_best_only=True,verbose=1)

callbacks = [checkpointer]

In [ ]:
history = model.fit(x_train,y_train,batch_size=32,epochs=70,verbose=1,callbacks=[checkpointer],validation_split=0.05)


In [ ]:
model = load_model('tesla.stock_prediction.hdf5')


In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

fig, ax = plt.subplots(figsize=(35,10))
plt.plot(df, color='red',  label="True Stock Price",)
ax.plot(range(len(y_train)+50,len(y_train)+50+len(predictions)),predictions, color='blue', label='Predicted Stock Price')
plt.legend()

In [ ]:
y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))


fig, ax = plt.subplots(figsize=(35,10))
ax.plot(y_test_scaled, color='red', label='True Stock Price')
plt.plot(predictions, color='blue', label='Predicted Stock Price')
plt.legend()